In [52]:
import copy
import requests
import time as old_time
from dateutil.parser import parse as date_parse 
import numpy as np
import pandas as pd
import csv
from pathlib import Path
import urllib
import json
import pytz
from typing import NamedTuple
from io import StringIO
from yaml import load
try:
    from yaml import CLoader as Loader
except ImportError:
    from yaml import Loader

from astropy.coordinates import EarthLocation
from astroplan import Observer, AtNightConstraint
import astropy.units as u
from astropy.coordinates import SkyCoord, Angle
from astropy.time import Time
from astroplan import FixedTarget, EclipsingSystem, LocalTimeConstraint, is_event_observable, is_always_observable, AltitudeConstraint
from scipy.interpolate import RegularGridInterpolator

from astroplan import Constraint
from astroplan.constraints import _get_altaz

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

Load Exoclock datafile - do this once and then use the locally picked version

In [4]:
js = urllib.request.urlopen('https://www.exoclock.space/database/planets_json').read().decode()
exoclock_t = pd.read_json(StringIO(js))
exoclock = exoclock_t.transpose()
exoclock.to_pickle("exoclock.pickle")

If you have the picked exoclock data execute this instead

In [5]:
exoclock = pd.read_pickle("exoclock.pickle")

Set up the observatory and the telescope

In [8]:
class Telescope(NamedTuple):
    fl_mm: float
    aper_mm: float
location = EarthLocation.from_geodetic(lon=-116.865*u.deg, lat=33.3564*u.deg, height=1712*u.m)
observer = Observer(location=location, name="Palomar", timezone="US/Pacific", 
                   temperature=12*u.deg_C, pressure=985.0*u.hPa, relative_humidity=.18)
telescope = Telescope(fl_mm=16750, aper_mm=5100)  # yeah, a monster

Some useful time and unit functions

In [144]:
def to_utc(observer: Observer, dt: Time, location: EarthLocation) -> Time:
    tz = observer.timezone
    ndt = tz.normalize(tz.localize(dt.to_datetime())).astimezone(pytz.utc)
    return Time(ndt, location=location)

def to_local(observer: Observer, dt: Time, location: EarthLocation) -> Time:
    tzutc = pytz.utc
    ndt = tzutc.localize(dt.to_datetime())
    lt = ndt.astimezone(observer.timezone)
    return Time(lt.replace(tzinfo=None), location=location)

def exou_to_u(exo_unit: str) -> u.Unit:
    if exo_unit == "Days":
        return u.day
    elif exo_unit == "Hours":
        return u.hour
    elif exo_unit == "Seconds":
        return u.second
    elif exo_unit == "Degrees":
        return u.deg
    elif exo_unit == "Radians":
        return u.rad
    else:
        raise ValueError(f"Unsupported unit: {exo_unit}")
    
def exou_tf_to_u(time_string: str, format: str, location: EarthLocation) -> Time:
    format, scale = format.split("_")
    if format == "BJD" or format == "JD":
        f = "jd"
    elif format == "MJD":
        f = "mjd"
    else:
        raise ValueError(f"Unknown time format: {format}")
    s = scale.lower()
    if s == 'local':
        s = 'Local'
    return Time(time_string, format=f, scale=s, location=location)

In [142]:
t = Time("2025-01-01 10:11:12", location=location)
print(t.iso)
ut = to_utc(observer, t)
print(ut.iso)
lt = to_local(observer, ut)
print(lt.iso)

2025-01-01 10:11:12.000
2025-01-01 18:11:12.000
2025-01-01 10:11:12.000


Observation date/time (_in local time_) which will then be converted to UTC

In [146]:
start_time = to_utc(observer, Time("2025-01-01T16:00:00.0", location=location), location)
end_time = to_utc(observer, Time("2025-01-02T07:00:00.0", location=location), location)

Filter off all exoclock targets based on telescope aperture

In [147]:
possible_targets = exoclock[exoclock['min_telescope_inches'] <= (telescope.aper_mm*u.mm).to(u.imperial.inch).value]

In [187]:
possible_targets['ephem_mid_time_format'].unique()

array(['BJD_TDB'], dtype=object)

Create eclipsing system for each possible target and then get the list of transits that will happen between start and end times

In [150]:
eclipsing_systems = [
    (
        EclipsingSystem(primary_eclipse_time=exou_tf_to_u(row['ephem_mid_time'], row['ephem_mid_time_format'], location),
                        orbital_period=row['ephem_period'] * exou_to_u(row['ephem_period_units']),
                        duration=row['duration_hours'] * u.hour,
                        name=row['name'],
                        eccentricity=row['eccentricity'],
                        argument_of_periapsis=row['periastron'] * exou_to_u(row['periastron_units'])),
        FixedTarget(SkyCoord(f"{row['ra_j2000']} {row['dec_j2000']}", unit=(u.hourangle, u.deg)))
    )
    for _, row 
    in possible_targets.iterrows()
    ]


In [151]:
transits = [
    es.next_primary_eclipse_time(start_time, n_eclipses=np.ceil(((end_time-start_time)/es.period).value))
    for es, _
    in eclipsing_systems
]


Now create constraints for visibility

We will start with only using horizon > 20 deg as the other time constraints will be dealt with by time filtering later on.

In [152]:
constraints = [
    AltitudeConstraint(min=20*u.deg),
    
]

In [153]:
is_observable = [
    is_event_observable(constraints, observer, ecs[1], mid_transit_times[0])
    for ecs, mid_transit_times
    in zip(eclipsing_systems, transits)
]

Now, pick only the systems that are observable

In [169]:
visible = [
    (ecs[0].name, 
     (trans[0]-ecs[0].duration/2.0-1*u.hour), trans[0],(trans[0]+ecs[0].duration/2.0+1*u.hour), 
     to_local(observer, trans[0], location).iso, 
     ecs[0].duration.value)
    for ecs, trans, visible
    in zip(eclipsing_systems, transits, is_observable)
    if visible and start_time <= trans[0]-ecs[0].duration/2.0-1*u.hour <= trans[0]+ecs[0].duration/2.0+1*u.hour <= end_time
]

In [170]:
sl = sorted(visible, key=lambda e: e[1])

In [171]:
s = StringIO()
writer = csv.writer(s)
for r in sl:
    writer.writerow(r)
s

In [172]:
s.seek(0)
print(s.getvalue())

TOI-4087b,2025-01-02 00:09:58.039268,2025-01-02 02:39:22.039268,2025-01-02 05:08:46.039268,2025-01-01 18:39:22.039,2.98
TOI-2025b,2025-01-02 00:30:37.907246,2025-01-02 03:18:55.907246,2025-01-02 06:07:13.907246,2025-01-01 19:18:55.907,3.61
XO-4b,2025-01-02 00:35:41.383086,2025-01-02 03:48:35.383086,2025-01-02 07:01:29.383086,2025-01-01 19:48:35.383,4.43
WASP-12b,2025-01-02 01:48:51.039333,2025-01-02 04:18:51.039333,2025-01-02 06:48:51.039333,2025-01-01 20:18:51.039,3.0
TOI-1442b,2025-01-02 02:26:30.776390,2025-01-02 03:45:24.776390,2025-01-02 05:04:18.776390,2025-01-01 19:45:24.776,0.63
K2-141b,2025-01-02 02:29:07.750742,2025-01-02 03:58:31.750742,2025-01-02 05:27:55.750742,2025-01-01 19:58:31.751,0.98
WASP-72b,2025-01-02 02:32:06.811894,2025-01-02 05:27:36.811894,2025-01-02 08:23:06.811894,2025-01-01 21:27:36.812,3.85
TOI-544b,2025-01-02 03:19:42.262924,2025-01-02 04:54:48.262924,2025-01-02 06:29:54.262924,2025-01-01 20:54:48.263,1.17
TOI-2445b,2025-01-02 03:29:52.544650,2025-01-02 04

We appear to have mid-transit times that don't quite match any online observation planners. why?

In [173]:
sl[11]

('KELT-24b',
 <Time object: scale='utc' format='datetime' value=2025-01-02 04:04:03.905010>,
 <Time object: scale='utc' format='datetime' value=2025-01-02 07:14:33.905010>,
 <Time object: scale='utc' format='datetime' value=2025-01-02 10:25:03.905010>,
 '2025-01-01 23:14:33.905',
 4.35)

In [174]:
def find_loc(name: str) -> EarthLocation:
    for e in eclipsing_systems:
        if e[0].name == name:
            return e[1]
    return None

In [177]:
kelt = find_loc("KELT-24b")
kelt

<FixedTarget "None" at SkyCoord (ICRS): (ra, dec) in deg (161.90979583, 71.65587667)>

In [183]:
sl[11][2].light_travel_time(kelt.coord, 'barycentric').to(u.minute)

<Quantity 4.23679254 min>

In [185]:
sl[-1]

('TOI-1693b',
 <Time object: scale='utc' format='datetime' value=2025-01-02 10:40:09.526396>,
 <Time object: scale='utc' format='datetime' value=2025-01-02 12:17:57.526396>,
 <Time object: scale='utc' format='datetime' value=2025-01-02 13:55:45.526396>,
 '2025-01-02 04:17:57.526',
 1.26)

In [186]:
toi = find_loc('TOI-1693b')
sl[-1][2].light_travel_time(toi.coord, 'barycentric').to(u.minute)

<Quantity 7.81102036 min>

The difference is *exactly* the heliocentric vs barycentric!!!

Our calculations appear to be barycentric as they are + light travel times while online tools give us heliocentric times. 
`astroplan` has a big warning that it does not do any barycentric corrections and that some of the calcs will be sus because of it.